<a href="https://colab.research.google.com/github/arunavijayan2299/Study_Assistant/blob/main/Study_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
LLM_Multi-Agent Study Assistant
File: LLM_Multi-Agent_Study_Assistant.py

A self-contained Python implementation (single-file) of an LLM-powered
Multi-Agent Study Assistant designed for the Kaggle capstone in the Study/Enterprise
Agents style. The notebook demonstrates:
- Multi-agent orchestration (GoalAgent, PlannerAgent, QuizAgent, ReviewAgent, Coordinator)
- Tools: ProgressStore (custom tool) and optional CalendarTool stub
- Sessions & Memory: SessionService and MemoryBank (in-memory + file-backed)
- Observability: structured logging and simple metrics
- Agent evaluation: simple user-simulation tests and performance tracking

USAGE:
- Runs in MOCK mode (no external API calls) by default so it is safe to run in
  Kaggle notebooks.
- To enable a real LLM, implement the LLM.call method (OpenAI, Google, etc.)
  and provide credentials as environment variables.

Features showcased:
- Multi-agent flows: goal analysis -> planning -> content generation (quizzes/flashcards)
- Memory persistence: tracks student progress and difficulty
- Long-running ops: simulated spaced repetition scheduling
- Observability: metrics for generated quizzes and review suggestions

Run this file to execute the demo and unit tests.
"""

from __future__ import annotations
import json
import os
import re
import time
import uuid
import logging
from typing import Dict, Any, List, Optional

# ---------------------------
# Observability
# ---------------------------
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')
logger = logging.getLogger('multiagent-study')

metrics = {
    'total_requests': 0,
    'quizzes_generated': 0,
    'reviews_scheduled': 0,
}

# ---------------------------
# Simple LLM interface and MockLLM
# ---------------------------
class LLM:
    def call(self, prompt: str, system: Optional[str] = None) -> str:
        raise NotImplementedError

class MockLLM(LLM):
    """Heuristic mock LLM used for demonstration and tests."""
    def call(self, prompt: str, system: Optional[str] = None) -> str:
        txt = (system or '') + '\n' + prompt
        txt = txt.lower()
        # Goal analysis
        if 'analyze goals' in (system or '').lower() or 'what are the study goals' in prompt.lower():
            # find exam/topic/duration keywords
            topic = 'general'
            days = 7
            match = re.search(r'exam|exam in (\d+)', prompt.lower())
            if 'final' in txt:
                topic = 'finals'
                days = 14
            num_match = re.search(r'(\d+) days', prompt.lower())
            if num_match:
                days = int(num_match.group(1))
            return json.dumps({'topic': topic, 'days_until_exam': days, 'priority': 'high' if days < 7 else 'medium'})
        # Planner
        if 'create plan' in (system or '').lower() or 'generate schedule' in prompt.lower():
            # simple schedule: distribute sessions across days
            plan = []
            for i in range(1,6):
                plan.append({'day': i, 'sessions': [f'Read chapter {i}', f'Practice problems {i}']})
            return json.dumps({'plan': plan})
        # Quiz generation
        if 'generate quiz' in (system or '').lower() or 'make a quiz' in prompt.lower():
            metrics['quizzes_generated'] += 1
            q = [
                {'q': 'What is the definition of X?', 'a': 'Definition of X'},
                {'q': 'List 3 examples of Y', 'a': 'example1; example2; example3'}
            ]
            return json.dumps({'quiz': q})
        # Flashcards / review
        if 'create flashcards' in (system or '').lower() or 'flashcards' in prompt.lower():
            return json.dumps({'cards': [{'front': 'Term A', 'back': 'Definition A'}, {'front': 'Term B', 'back': 'Definition B'}]})
        # Scheduling review
        if 'schedule review' in (system or '').lower() or 'when should they review' in prompt.lower():
            metrics['reviews_scheduled'] += 1
            return json.dumps({'next_review_days': 2})
        # Fallback
        return 'I do not understand.'

# ---------------------------
# Sessions & Memory
# ---------------------------
class SessionService:
    def __init__(self):
        self.sessions: Dict[str, List[Dict[str, Any]]] = {}

    def new_session(self) -> str:
        sid = str(uuid.uuid4())
        self.sessions[sid] = []
        logger.debug(f'New session: {sid}')
        return sid

    def append_message(self, session_id: str, role: str, content: str):
        if session_id not in self.sessions:
            raise KeyError('Unknown session')
        self.sessions[session_id].append({'role': role, 'content': content, 'ts': time.time()})

    def get_session(self, session_id: str) -> List[Dict[str, Any]]:
        return self.sessions.get(session_id, [])

class MemoryBank:
    def __init__(self, path: Optional[str] = None):
        self.mem: Dict[str, Dict[str, Any]] = {}
        self.path = path
        if path and os.path.exists(path):
            try:
                with open(path, 'r') as f:
                    self.mem = json.load(f)
                    logger.info('MemoryBank loaded')
            except Exception as e:
                logger.warning('Failed to load memory: ' + str(e))

    def get(self, student_id: str) -> Dict[str, Any]:
        return self.mem.get(student_id, {})

    def update(self, student_id: str, data: Dict[str, Any]):
        existing = self.mem.get(student_id, {})
        existing.update(data)
        self.mem[student_id] = existing
        if self.path:
            with open(self.path, 'w') as f:
                json.dump(self.mem, f, indent=2)

# ---------------------------
# Tools
# ---------------------------
class ProgressStore:
    def __init__(self, path: str = 'progress.json'):
        self.path = path
        if not os.path.exists(self.path):
            with open(self.path, 'w') as f:
                json.dump([], f)

    def save_progress(self, record: Dict[str, Any]) -> str:
        with open(self.path, 'r') as f:
            data = json.load(f)
        rec_id = str(uuid.uuid4())
        record['id'] = rec_id
        data.append(record)
        with open(self.path, 'w') as f:
            json.dump(data, f, indent=2)
        logger.info(f'Progress saved: {rec_id}')
        return rec_id

# ---------------------------
# Agents
# ---------------------------
class GoalAgent:
    def __init__(self, llm: LLM):
        self.llm = llm

    def analyze(self, prompt: str) -> Dict[str, Any]:
        system = 'Analyze goals: identify topic, days_until_exam, and priority. Return JSON.'
        raw = self.llm.call(prompt, system=system)
        try:
            res = json.loads(raw)
            logger.info(f'GoalAgent: {res}')
            return res
        except Exception:
            logger.info('GoalAgent fallback')
            return {'topic': 'general', 'days_until_exam': 7, 'priority': 'medium'}

class PlannerAgent:
    def __init__(self, llm: LLM):
        self.llm = llm

    def create_plan(self, topic: str, days: int) -> Dict[str, Any]:
        system = 'Create plan: generate a study plan JSON distributing topics across days.'
        prompt = f'Topic: {topic}\nDays: {days}\nGenerate schedule plan as JSON.'
        raw = self.llm.call(prompt, system=system)
        try:
            plan = json.loads(raw)
            logger.info('PlannerAgent created plan')
            return plan
        except Exception:
            logger.info('PlannerAgent fallback')
            return {'plan': [{'day': 1, 'sessions': ['Intro', 'Practice']}]}

class QuizAgent:
    def __init__(self, llm: LLM):
        self.llm = llm

    def generate_quiz(self, topic: str, difficulty: str = 'medium') -> Dict[str, Any]:
        system = 'Generate quiz: produce JSON with a list of Q/A pairs.'
        prompt = f'Create a short quiz for {topic} at {difficulty} difficulty.'
        raw = self.llm.call(prompt, system=system)
        try:
            quiz = json.loads(raw)
            logger.info('QuizAgent generated quiz')
            return quiz
        except Exception:
            logger.info('QuizAgent fallback')
            return {'quiz': [{'q': 'Sample Q?', 'a': 'Sample A'}]}

class ReviewAgent:
    def __init__(self, llm: LLM):
        self.llm = llm

    def schedule_review(self, performance: Dict[str, Any]) -> Dict[str, Any]:
        system = 'Schedule review: return JSON {next_review_days: int}.'
        prompt = f'Performance: {json.dumps(performance)}\nWhen should they review?'
        raw = self.llm.call(prompt, system=system)
        try:
            res = json.loads(raw)
            logger.info('ReviewAgent scheduled review')
            return res
        except Exception:
            logger.info('ReviewAgent fallback')
            return {'next_review_days': 2}

# ---------------------------
# Coordinator
# ---------------------------
class Coordinator:
    def __init__(self, llm: LLM, progress_store: ProgressStore, memory_bank: MemoryBank, sessions: SessionService):
        self.goal_agent = GoalAgent(llm)
        self.planner = PlannerAgent(llm)
        self.quiz_agent = QuizAgent(llm)
        self.review_agent = ReviewAgent(llm)
        self.progress_store = progress_store
        self.memory_bank = memory_bank
        self.sessions = sessions

    def handle_request(self, user_prompt: str, student_id: Optional[str] = None, session_id: Optional[str] = None) -> Dict[str, Any]:
        metrics['total_requests'] += 1
        logger.info('Coordinator received study request')
        if session_id is None:
            session_id = self.sessions.new_session()
        self.sessions.append_message(session_id, 'user', user_prompt)

        # Goal analysis
        goal = self.goal_agent.analyze(user_prompt)
        topic = goal.get('topic', 'general')
        days = int(goal.get('days_until_exam', 7))

        # Planning
        plan = self.planner.create_plan(topic, days)

        # Generate initial quiz and flashcards
        quiz = self.quiz_agent.generate_quiz(topic)
        flashcards = {'cards': [{'front': 'Term X', 'back': 'Definition X'}]}

        # Save progress/tool usage
        record = {
            'student_id': student_id,
            'topic': topic,
            'plan_summary': plan,
            'quiz_summary': quiz,
            'ts': time.time(),
        }
        rec_id = self.progress_store.save_progress(record)

        # Update memory
        if student_id:
            existing = self.memory_bank.get(student_id)
            contacts = existing.get('sessions', 0) + 1
            self.memory_bank.update(student_id, {'last_plan_id': rec_id, 'sessions': contacts, 'topic': topic})

        res = {
            'session_id': session_id,
            'student_id': student_id,
            'goal': goal,
            'plan': plan,
            'quiz': quiz,
            'flashcards': flashcards,
            'progress_record_id': rec_id,
        }
        logger.info('Coordinator completed study orchestration')
        return res

# ---------------------------
# Evaluation harness & demo
# ---------------------------

def evaluate_system(coord: Coordinator, tests: List[Dict[str, Any]]) -> Dict[str, Any]:
    results = []
    for t in tests:
        out = coord.handle_request(t['prompt'], student_id=t.get('student_id'))
        results.append({'prompt': t['prompt'], 'goal': out['goal'], 'plan_exists': bool(out['plan'])})
    logger.info('Evaluation complete')
    return {'results': results}


def demo_mode(llm: LLM):
    progress = ProgressStore('demo_progress.json')
    memory = MemoryBank('demo_study_memory.json')
    sessions = SessionService()
    coord = Coordinator(llm, progress, memory, sessions)

    samples = [
        {'prompt': 'I have a math final in 10 days, help me study derivatives', 'student_id': 's1'},
        {'prompt': 'I need to prepare for a history exam in 5 days covering WW2', 'student_id': 's2'},
    ]

    eval_res = evaluate_system(coord, samples)
    print('\n=== Evaluation ===')
    print(json.dumps(eval_res, indent=2))
    print('\n=== Metrics ===')
    print(json.dumps(metrics, indent=2))
    print('\n=== Memory ===')
    print(json.dumps(memory.mem, indent=2))

# ---------------------------
# OpenAIStub placeholder
# ---------------------------
class OpenAIStub(LLM):
    def __init__(self, api_key: Optional[str] = None):
        self.api_key = api_key or os.environ.get('OPENAI_API_KEY')

    def call(self, prompt: str, system: Optional[str] = None) -> str:
        raise RuntimeError('Replace OpenAIStub.call with your provider API code.')

# ---------------------------
# Main
# ---------------------------
if __name__ == '__main__':
    print('Starting LLM Multi-Agent Study Assistant demo (MOCK LLM)')
    llm = MockLLM()
    demo_mode(llm)
    print('\nDemo complete. To use a real LLM, implement OpenAIStub.call or another LLM wrapper and re-run.')

# ---------------------------
# README (for Kaggle submission)
# ---------------------------
README = '''
Project: LLM Multi-Agent Study Assistant

Overview
--------
This project demonstrates a study assistant built as a coordinated multi-agent
system. Agents include GoalAgent (understands student goals), PlannerAgent
(generates schedules), QuizAgent (creates quizzes), and ReviewAgent
(schedules spaced repetition). A Coordinator orchestrates agents and stores
progress in a simple tool.

How to run
----------
- Run this file in a Python environment; it runs with a MockLLM by default.
- To enable a real model, implement an LLM subclass with your provider calls.

Features demonstrated
---------------------
- Multi-agent orchestration
- Custom tool: ProgressStore
- Sessions & MemoryBank
- Metrics & lightweight evaluation
- Extensible design for real LLM integration

Future work
-----------
- Swap MockLLM for a real LLM and fine-tune prompts
- Add richer flashcard generation and spaced-repetition algorithm
- Integrate calendar APIs and push notifications
- Add evaluation against student performance data
'''

with open('README_STUDY.md', 'w') as f:
    f.write(README)



Starting LLM Multi-Agent Study Assistant demo (MOCK LLM)

=== Evaluation ===
{
  "results": [
    {
      "prompt": "I have a math final in 10 days, help me study derivatives",
      "goal": {
        "topic": "finals",
        "days_until_exam": 10,
        "priority": "medium"
      },
      "plan_exists": true
    },
    {
      "prompt": "I need to prepare for a history exam in 5 days covering WW2",
      "goal": {
        "topic": "general",
        "days_until_exam": 5,
        "priority": "high"
      },
      "plan_exists": true
    }
  ]
}

=== Metrics ===
{
  "total_requests": 2,
  "quizzes_generated": 2,
  "reviews_scheduled": 0
}

=== Memory ===
{
  "s1": {
    "last_plan_id": "877f1aa4-3472-464b-ae25-41bc28ddfaa2",
    "sessions": 3,
    "topic": "finals"
  },
  "s2": {
    "last_plan_id": "e568482a-b7a7-42ca-8c48-18240ed9dc47",
    "sessions": 3,
    "topic": "general"
  }
}

Demo complete. To use a real LLM, implement OpenAIStub.call or another LLM wrapper and re-run.
